In [1]:
print("Hello")

Hello


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']

## Campletion Model

In [3]:
from langchain_groq import ChatGroq
llamaChatModel = ChatGroq(
    model="llama3-70b-8192"
)

## Prompt & Prompt Template

In [7]:
# This is for Completions Model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    'Tell me a {adjective} story about {topic}'
)

llmModelPromt = prompt_template.format(
    adjective = "curious",
    topic = 'mountain'
)

res = llamaChatModel.invoke(llmModelPromt)

In [8]:
print(res.content)

Here's a curious story about a mountain:

**The Mysterious Hum of Mount Tamalpais**

In the heart of Marin County, California, stands Mount Tamalpais, a majestic mountain with a rich cultural and spiritual significance to the indigenous Coast Miwok people. For centuries, the mountain has been revered for its breathtaking beauty, diverse wildlife, and... its strange, unexplained hum.

Locals and visitors alike have reported hearing a low, steady hum, often described as a gentle, pulsing drone, emanating from the mountain. The sound is usually heard at dawn and dusk, when the air is still, and can be loud enough to be felt as much as heard. Some claim it's like the mountain is "singing" or "vibrating" with an otherworldly energy.

The first recorded mention of the hum dates back to the 1800s, when Native American tribes would gather at the mountain's base to perform rituals and ceremonies. They believed the hum was the mountain's way of communicating with them, sharing ancient wisdom and

## Chat Completion Model

In [15]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ('system', "You are an {profession} expert on {topic}."),
    ('human', 'Hello Mr.{profession}, can you please answer a question?'),
    ('ai', 'Sure!'),
    ("human", '{user_input}'),
])

messages = chat_template.format_messages(
    profession = "Historian",
    topic = 'The Kennedy family',
    user_input ='How many grandchildren had Joseph P. Kennedy?' 
)

In [16]:
res = llamaChatModel.invoke(messages)

In [17]:
print(res.content)

Joseph P. Kennedy, the patriarch of the Kennedy family, had a total of 29 grandchildren. He had nine children with his wife Rose Fitzgerald Kennedy, and those children went on to have many children of their own. The 29 grandchildren of Joseph P. Kennedy include many notable figures, such as John F. Kennedy Jr., Caroline Bouvier Kennedy, Maria Shriver, and Patrick Kennedy, among others.


## Few Shot Prompting

In [19]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

exemple = [
    {"input": 'Hi!', 'output': 'Hola'},
    {"input": 'Bey!', 'output': 'adios'}
]

exemple_template = ChatPromptTemplate.from_messages(
    [
        ('human', '{input}'),
        ('ai', '{output}'),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=exemple_template,
    examples=exemple
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',  'You are an English-Spanish translator'),
        few_shot_prompt,
        ('human', '{input}')
    ]
)


## Chains

In [ ]:
chain = final_prompt | llamaChatModel # LangChain Expression Language

res = chain.invoke({'input': "How are you?"})

print(res.content)

Estoy bien, ¿y tú?


## Output Parsers For Response

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an 'answer' key that answers the following question : {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llamaChatModel | json_parser

In [5]:
json_chain.invoke({'question': "What is the biggest country?"})

{'answer': 'Russia'}

#### Optionally we can use Pydantic to define a custom output format

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1  import BaseModel, Field

In [9]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup : str = Field(description="question to set up a joke")
    punchline : str = Field(description="answer to resolve the joke")

In [ ]:
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template= "Answer the user query. \n{format_instruction}\n{query}\n",
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()} ## answer base on the Custom format
)

chain = prompt | llamaChatModel | parser
chain.invoke({"query": "Tell me a Joke. "})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything.'}